In [56]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
# import warnings

In [57]:
df=pd.read_csv('studeval.csv',sep=',')

In [58]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [59]:
df['total_score']= df['math_score']+df['reading_score']+df['writing_score']
df['average_score']= df['total_score']/3

In [60]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,average_score
0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,female,group C,some college,standard,completed,69,90,88,247,82.333333
2,female,group B,master's degree,standard,none,90,95,93,278,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,49.333333
4,male,group C,some college,standard,none,76,78,75,229,76.333333


In [61]:
x=df.drop('average_score',axis=1)
y=df['average_score']

In [62]:
num_feat=[feat for feat in x.columns if df[feat].dtype!='O']
cat_feat=[feat for feat in x.columns if df[feat].dtype=='O']

In [63]:
print(num_feat)
print(cat_feat)

['math_score', 'reading_score', 'writing_score', 'total_score']
['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch', 'test_preparation_course']


In [64]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

ohe=OneHotEncoder()
scaler=StandardScaler()

col_tras=ColumnTransformer(
  [
    ("OneHotEncoding",ohe,cat_feat),
    ("StandardScaling",scaler,num_feat)
  ]
)

In [65]:
x=col_tras.fit_transform(x)

In [66]:
x

array([[ 1.        ,  0.        ,  0.        , ...,  0.19399858,
         0.39149181,  0.34357423],
       [ 1.        ,  0.        ,  0.        , ...,  1.42747598,
         1.31326868,  1.0219275 ],
       [ 1.        ,  0.        ,  0.        , ...,  1.77010859,
         1.64247471,  1.74706375],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.12547206,
        -0.20107904, -0.19443008],
       [ 1.        ,  0.        ,  0.        , ...,  0.60515772,
         0.58901542,  0.46053169],
       [ 1.        ,  0.        ,  0.        , ...,  1.15336989,
         1.18158627,  1.06871048]])

In [67]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape, x_test.shape

((800, 21), (200, 21))

In [68]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [69]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

In [72]:
mdls=[]
r2_scores=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    y_test_pred=model.predict(x_test)
    y_train_pred=model.predict(x_train)

    mae_train, rmse_train, r2_square_train = evaluate_model(y_train, y_train_pred)

    mae, rmse, r2_square = evaluate_model(y_test, y_test_pred)


    print("model name",list(models.keys())[i])
    mdls.append(list(models.keys())[i])


    print("Train data")
    print("MAE:{:.4f}".format(mae_train))
    print("RMSE:{:.4f}".format(rmse_train))
    print("R2 Square:{:.4f}".format(r2_square_train))

    print('----------------------------------')

    print("Test data")
    print("MAE:{:.4f}".format(mae))
    print("RMSE:{:.4f}".format(rmse))
    print("R2 Square:{:.4f}".format(r2_square))
    print("\n")

    r2_scores.append(r2_square)

model name Linear Regression
Train data
MAE:0.0000
RMSE:0.0000
R2 Square:1.0000
----------------------------------
Test data
MAE:0.0000
RMSE:0.0000
R2 Square:1.0000


model name Lasso
Train data
MAE:0.8075
RMSE:1.0090
R2 Square:0.9949
----------------------------------
Test data
MAE:0.8312
RMSE:1.0557
R2 Square:0.9948


model name Ridge
Train data
MAE:0.0048
RMSE:0.0059
R2 Square:1.0000
----------------------------------
Test data
MAE:0.0049
RMSE:0.0064
R2 Square:1.0000


model name K-Neighbors Regressor
Train data
MAE:1.5570
RMSE:1.9816
R2 Square:0.9803
----------------------------------
Test data
MAE:1.9230
RMSE:2.6661
R2 Square:0.9668


model name Decision Tree
Train data
MAE:0.0000
RMSE:0.0000
R2 Square:1.0000
----------------------------------
Test data
MAE:0.0900
RMSE:0.6831
R2 Square:0.9978


model name Random Forest Regressor
Train data
MAE:0.0254
RMSE:0.1445
R2 Square:0.9999
----------------------------------
Test data
MAE:0.1221
RMSE:0.9492
R2 Square:0.9958


model name XGBRe

In [74]:
pd.DataFrame(list(zip(mdls,r2_scores)),columns=['models','r2_score']).sort_values(by=['r2_score'],ascending=False)

,models,r2_score
0,Linear Regression,1.000000
2,Ridge,1.000000
4,Decision Tree,0.997823
6,XGBRegressor,0.997431
5,Random Forest Regressor,0.995797
7,CatBoosting Regressor,0.994961
1,Lasso,0.994801
8,AdaBoost Regressor,0.990354
3,K-Neighbors Regressor,0.966842


In [75]:
dec=DecisionTreeRegressor()
dec.fit(x_train,y_train)


DecisionTreeRegressor()

In [76]:
pred=dec.predict(x_test)

In [78]:
score = r2_score(y_test,pred)*100
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 99.75
